**NY STATE ENERGY CONSUMPTION DATA**
- Data in thsnd tons 
- API call from eia.gov
- sent to mysql workbench (creating tables and inserting data)

In [ ]:
import json
import requests
import time
import mysql.connector 
import pandas as pd
from datetime import date, datetime, timedelta
from functools import reduce
import mysql.connector 
from mysql.connector import errorcode

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [3]:
#checking connection to db
#config file in directory
import config

In [19]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx)

In [20]:
cursor = cnx.cursor()

In [21]:
cursor.close()
cnx.close()

In [5]:
api_key = config.api_key

**API CALL - EIA.gov (html url)**

In [6]:
coal_monthly_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.CONS_TOT.COW-NY-98.M'
petliq_monthly_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.CONS_TOT.PEL-NY-98.M'
petcoke_monthly_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.CONS_TOT.PC-NY-98.M'
natgas_monthly_url = f'http://api.eia.gov/series/?api_key={api_key}&series_id=ELEC.CONS_TOT.NG-NY-98.M'

In [7]:
response = requests.get(coal_monthly_url)
coal_md = response.json()

{'request': {'command': 'series', 'series_id': 'ELEC.CONS_TOT.COW-NY-98.M'},
 'series': [{'series_id': 'ELEC.CONS_TOT.COW-NY-98.M',
   'name': 'Total consumption : coal : New York : electric power (total) : monthly',
   'units': 'thousand tons',
   'f': 'M',
   'description': 'Summation of all types of coal; Power plants owned by companies whose primary purpose is to produce power; ',
   'copyright': 'None',
   'source': 'EIA, U.S. Energy Information Administration',
   'iso3166': 'USA-NY',
   'geography': 'USA-NY',
   'start': '200101',
   'end': '201907',
   'updated': '2019-09-23T17:31:34-0400',
   'data': [['201907', 20.67],
    ['201906', 0.413],
    ['201905', 0],
    ['201904', 5.204],
    ['201903', 27.486],
    ['201902', 24.012],
    ['201901', 89.869],
    ['201812', 21.612],
    ['201811', 1.413],
    ['201810', 1.274],
    ['201809', 21.333],
    ['201808', 16.441],
    ['201807', 49.24],
    ['201806', 6.253],
    ['201805', 0.753],
    ['201804', 0],
    ['201803', 47.81

In [8]:
response = requests.get(petliq_monthly_url)
petliq_md = response.json()

response = requests.get(petcoke_monthly_url)
petcoke_md = response.json()

response = requests.get(natgas_monthly_url)
natgas_md = response.json()

In [9]:
coal_mdates = pd.DataFrame(coal_md["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x,'%Y%m'))
coal_mseries = pd.DataFrame(coal_md['series'][0]['data'])[1]

mon_coal_NY = pd.concat([coal_mdates, coal_mseries.reindex(coal_mdates.index)], axis=1)

In [10]:
petliq_mdates = pd.DataFrame(petliq_md["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x,'%Y%m'))
petliq_mseries = pd.DataFrame(petliq_md['series'][0]['data'])[1]

mon_petliq_NY = pd.concat([petliq_mdates, petliq_mseries.reindex(petliq_mdates.index)], axis=1)
mon_petliq_NY

,0,1
0,2019-07-01,50.20193
1,2019-06-01,12.14224
2,2019-05-01,24.06632
3,2019-04-01,23.42744
4,2019-03-01,26.74117
5,2019-02-01,98.74642
6,2019-01-01,412.86716
7,2018-12-01,NaN
8,2018-11-01,36.07378
9,2018-10-01,15.40310


In [11]:
petcoke_mdates = pd.DataFrame(petcoke_md["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x,'%Y%m'))
petcoke_mseries = pd.DataFrame(petcoke_md['series'][0]['data'])[1]

mon_petcoke_NY = pd.concat([petcoke_mdates, petcoke_mseries.reindex(petcoke_mdates.index)], axis=1)
mon_petcoke_NY

,0,1
0,2019-07-01,0.000
1,2019-06-01,0.000
2,2019-05-01,0.000
3,2019-04-01,0.000
4,2019-03-01,0.000
5,2019-02-01,0.000
6,2019-01-01,0.000
7,2018-12-01,0.000
8,2018-11-01,0.000
9,2018-10-01,0.000


In [12]:
natgas_mdates = pd.DataFrame(natgas_md["series"][0]['data'])[0].apply(lambda x: datetime.strptime(x,'%Y%m'))
natgas_mseries = pd.DataFrame(natgas_md['series'][0]['data'])[1]

mon_natgas_NY = pd.concat([natgas_mdates, natgas_mseries.reindex(natgas_mdates.index)], axis=1)
mon_natgas_NY

,0,1
0,2019-07-01,55865.759
1,2019-06-01,32673.142
2,2019-05-01,23871.364
3,2019-04-01,24542.277
4,2019-03-01,26947.314
5,2019-02-01,28521.780
6,2019-01-01,30743.751
7,2018-12-01,29514.163
8,2018-11-01,30422.783
9,2018-10-01,33418.465


In [13]:
data_frames = [mon_coal_NY, mon_petliq_NY, mon_petcoke_NY, mon_natgas_NY]

In [14]:
monthly_totalconsumpt_NY = reduce(lambda left, right: pd.merge(left, right, on=[0], how = 'outer'), data_frames)

In [51]:
monthly_totalconsumpt_NY.columns=['date', 'coal', 'petroleum_liq', 'petroleum_coke', 'natural_gas']

In [2]:
monthly_totalconsumpt_NY.head() #in thousand tons

NameError: name 'monthly_totalconsumpt_NY' is not defined

In [53]:
# export = monthly_totalconsumpt_NY.to_csv('data.csv')
monthly_totalconsumpt_NY.columns

Index(['date', 'coal', 'petroleum_liq', 'petroleum_coke', 'natural_gas'], dtype='object')

**CREATE TABLES in Database** 

In [67]:
db_name = 'energydb'

In [68]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cursor = cnx.cursor()

In [69]:
TABLES = {}
TABLES['NY_consumpt_m'] = (
    "CREATE TABLE NY_consumpt ("
    "  date date PRIMARY KEY,"
    "  coal REAL,"
    "  petroleum_liq REAL,"
    "  petroleum_coke REAL,"
    "  natural_gas REAL"
    ") ENGINE=InnoDB")

In [70]:
cnx.commit()

In [71]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table NY_consumpt_m: OK


**INSERT DATA INTO TABLES**

In [84]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cursor = cnx.cursor()

In [85]:
def send_to_sql(dflisty, cursor):
    """
    takes energy consumption data (monthly fuels) = coal, pet liq, pet coke, and natural gase 
    ***in thousand tons***
    """
    cursor.execute('USE energydb')
    for i, row in dflisty.iterrows():
        print(i)
        
        cursor.execute("""INSERT INTO NY_consumpt(
                                        date,
                                        coal,
                                        petroleum_liq,
                                        petroleum_coke,
                                        natural_gas
                                        )
        VALUES ('{}','{}','{}','{}','{}'
        )""".format(row['date'],
                    row['coal'],
                    row['petroleum_liq'],
                    row['petroleum_coke'],
                    row['natural_gas']))
        cnx.commit()

In [87]:
cursor.close()
cnx.close()

In [86]:
send_to_sql(monthly_totalconsumpt_NY, cursor)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
